<a href="https://colab.research.google.com/github/knkobayashi/Music_Classification/blob/master/Test_Demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, roc_curve
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.naive_bayes import GaussianNB
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.cluster import KMeans


from scipy.stats import kurtosis
from scipy.stats import skew
import csv

import tensorflow as tf
from tensorflow.keras import layers
from keras.models import Sequential
from keras.layers import Dense

!pip install pafy
!pip install youtube_dl
!pip install pydub
import pafy, librosa, youtube_dl, warnings
from sklearn.preprocessing import MinMaxScaler
from pydub import AudioSegment

In [0]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
df = pd.read_csv("/content/drive/My Drive/EE461P_Project/Data.csv")

In [0]:
# Data from Dataframe
X = df.drop(['genre'], axis=1)
y = df['genre']

# Scaling X Data
scale = StandardScaler()
x_scaled = pd.DataFrame(scale.fit_transform(X), columns = X.columns)

# Encoding Y Data to integer from 0-9
encoder = LabelEncoder()
y_enc = encoder.fit_transform(y)

#Performing LDA
lda = LDA()
x_lda = lda.fit_transform(x_scaled, y_enc)

x_train, x_test, y_train, y_test = train_test_split(x_scaled, y_enc, test_size=0.25, random_state=42)
x_train_lda, x_test_lda, y_train_lda, y_test_lda = train_test_split(x_lda, y_enc, test_size=0.25, random_state=42)

# User for iterations below
dataset = []
dataset.append([x_train, x_test, y_train, y_test])
dataset.append([x_train_lda, x_test_lda, y_train_lda, y_test_lda])
xtrain, xtest, ytrain, ytest = range(0,4)

# x_train.shape

#PCA
from sklearn.decomposition import PCA

pca = PCA(n_components=3,whiten=False);
pca.fit(x_scaled)
xPca = pca.transform(x_scaled)

In [0]:
#@title
def get_features(y, sr):
    # Features to concatenate in the final dictionary
    features = {'centroid': None, 'roloff': None, 'flux': None, 'rmse': None,
                'zcr': None, 'contrast': None, 'bandwidth': None, 'flatness': None, 'chroma_stft': None}
    
    # Count silence
    if 0 < len(y):
        y_sound, _ = librosa.effects.trim(y)
    features['sample_silence'] = len(y) - len(y_sound)

    # Using librosa to calculate the features
    features['chroma_stft']=librosa.feature.chroma_stft(y=y, sr=sr).ravel()
    features['centroid'] = librosa.feature.spectral_centroid(y, sr=sr).ravel()
    features['roloff'] = librosa.feature.spectral_rolloff(y, sr=sr,).ravel()
    features['zcr'] = librosa.feature.zero_crossing_rate(y).ravel()
    features['rmse'] = librosa.feature.rms(y).ravel()
    features['flux'] = librosa.onset.onset_strength(y=y, sr=sr).ravel()
    features['contrast'] = librosa.feature.spectral_contrast(y, sr=sr).ravel()
    features['bandwidth'] = librosa.feature.spectral_bandwidth(y, sr=sr).ravel()
    features['flatness'] = librosa.feature.spectral_flatness(y).ravel()
    
    # MFCC treatment
    mfcc = librosa.feature.mfcc(y, sr=sr, n_mfcc=13)
    for idx, v_mfcc in enumerate(mfcc):
        features['mfcc_{}'.format(idx)] = v_mfcc.ravel()
        
    # Get statistics from the vectors
    def get_moments(descriptors):
        result = {}
        for k, v in descriptors.items():
            result['{}_max'.format(k)] = np.max(v)
            result['{}_min'.format(k)] = np.min(v)
            result['{}_mean'.format(k)] = np.mean(v)
            result['{}_std'.format(k)] = np.std(v)
            result['{}_kurtosis'.format(k)] = kurtosis(v)
            result['{}_skew'.format(k)] = skew(v)
        return result
    
    dict_agg_features = get_moments(features)
    dict_agg_features['tempo'] = librosa.beat.tempo(y, sr=sr)[0]
    
    return dict_agg_features

In [0]:
def success(label,columns,sums):
    mostNum = np.amax(label)
    mostLb = label.index(mostNum)
    mostLabel = columns[mostLb]
    print("Most found label: " + mostLabel)
    print("Number of " + mostLabel + " found :" + str(mostNum))
    total = np.sum(label)
    print("Population of cluster: " + str(total))
    mostSum = sums[mostLb]
    mostAbility = round(100*mostNum/mostSum,1)
    print("Percentage of " + mostLabel + " found: " + str(mostAbility) + "%")  
    ability = round(100*mostNum/total, 1)
    print("Ability to distinguish " + mostLabel + " amoung other found genres in cluster: " + str(ability)+"%")

def compareLabels(clusterLabels,trueLabels,labelNum,columns,plot=0):
    score = 0
    sums = [0]*10
    for k in trueLabels:
        sums[k] +=1
    for i in range(labelNum):
        if i ==0:
            clusters = [clusterLabels.index[clusterLabels[0]==i].to_numpy()]
            label = [0]*10
            for x in clusters[i]:
                label[trueLabels[x]] += 1
            labels = [label]
        else:
            clusters.append(clusterLabels.index[clusterLabels[0]==i].to_numpy())
            label = [0]*10
            for x in clusters[i]:
                label[trueLabels[x]] += 1
            labels.append(label)
        if plot==1:
            plt.figure(figsize=(8, 4))
            plt.bar(columns,label)
            title = "Match for cluster label " + str(i)
            plt.title(title)
            plt.show()
            success(labels[i],columns,sums)
    return clusters,labels

In [0]:
# Creating/Training Model
gnb = GaussianNB()
gnb.fit(dataset[1][xtrain], dataset[1][ytrain])

kmeans = KMeans(n_clusters=10,random_state=42)
kmeans.fit(xPca)
columns = y.unique()
clusterLabels = pd.DataFrame(kmeans.labels_)
clusters,labels = compareLabels(clusterLabels,y_enc,kmeans.n_clusters,columns)

In [0]:
# All-Star - https://www.youtube.com/watch?v=5ZYgIrqELFw
# Livin on a Prayer - https://www.youtube.com/watch?v=fDjhZva-6LE
# Through the Fire and Flames - https://www.youtube.com/watch?v=Wbrrma9Cutk
# Lose Yourself - https://www.youtube.com/watch?v=tR1ECf4sEpw
# September - https://www.youtube.com/watch?v=wBNewLDy3pQ

def download_songs (video_url, song_offset):
  with warnings.catch_warnings():
    warnings.filterwarnings("ignore",category=DeprecationWarning)

    # Getting the video
    video = pafy.new(url=video_url, basic=False, gdata=False)
    length = video.length
    stream = video.getbestaudio()
    filename = stream.download()
    print("Downloaded: '" + video.title + '.' + stream.extension + "'")

    # Convert to MP3
    name = 'song.mp3'
    wav_audio = AudioSegment.from_file((video.title + '.' + stream.extension), format=stream.extension)
    audio = wav_audio.export(name, format="mp3")
    print("Converted into: '" + name + "'")

    # Extract Features
    y, sr = librosa.load(name, offset=song_offset, duration = 60)
    features = get_features(y, sr)
    features['genre'] = 'hiphop'
    features['title'] = video.title
    features = pd.DataFrame(features, index=[0])
    print("Successfully Extracted Features")
    return features

def supervised_from_youtube ( song ):
    # Predicting Model
    genres = ['blues', 'classical', 'country', 'disco', 'hiphop', 'jazz', 'metal', 'pop', 'reggae', 'rock']
    song_x = scale.transform(song.drop(['genre'], axis=1).drop(['title'], axis = 1))
    song_x = lda.transform(song_x)
    song_y = song['genre']

    prediction = gnb.predict(song_x)
    predict_proba = gnb.predict_log_proba(song_x)

    proba_genre = dict(zip(genres, predict_proba[0]))
    proba_genre_sorted = sorted(proba_genre.items(), key=lambda x: x[1], reverse=True)

    # Printing Results
    print("================================")
    print("Supervised Predicted '" + song.title[0] + "' as " + str(encoder.inverse_transform(prediction)[0]).title())
    print("Guess order is: ", end="")
    for x in proba_genre_sorted: print(x[0].title(), end=", ")
    print("")

def unsupervised_from_youtube( song ):
    # Predicting Model
    genres = ['blues', 'classical', 'country', 'disco', 'hiphop', 'jazz', 'metal', 'pop', 'reggae', 'rock']
    song_x = scale.transform(song.drop(['genre'], axis=1).drop(['title'], axis = 1))
    song_x = pca.transform(song_x)

    prediction = kmeans.predict(song_x)
    k = np.squeeze(prediction)

    proba_genre = dict(zip(genres, labels[k]))
    proba_genre_sorted = sorted(proba_genre.items(), key=lambda x: x[1], reverse=True)

    # Printing Results
    print("================================")
    #print("Unsupervised Predicted '" + song.title[0] + "' as " + str(proba_genre_sorted[0][0]).title())
    #print("Guess order is: ", end="")
    #for x in proba_genre_sorted: print(x[0].title(), end=", ")
    #print("")
    print("Kmeans Group: " + str(kmeans.predict(song_x)[0]))
    print("")

In [0]:
#Pre Chosen Songs:

#Simple Examples:
moanin_jazz = download_songs("https://www.youtube.com/watch?v=Cv9NSR-2DwM", 0) #Art Blakely's Moanin - A Jazz Standard
requiem_d_classic = download_songs("https://www.youtube.com/watch?v=QB0877E14mo", 0) #Mozarts requiem in D-Minor from watchmen - A modern arrangement of a classic piece
three_little_birds_marley = download_songs("https://www.youtube.com/watch?v=HNBCVM4KbUM", 0) #Bob Marley's Three Little Birds - An Iconic Reggae Song
big_iron_country = download_songs("https://www.youtube.com/watch?v=zzICMIu5zFY", 30) #Marty Robin's Big Iron - Country Song
master_of_puppets_metallica = download_songs("https://www.youtube.com/watch?v=xnKhsTXoKCI", 90) #Metallica's Master of Puppets - A chart-topping Metal Song
sweet_child_roses = download_songs("https://www.youtube.com/watch?v=1w7OgIMMRc4", 30) #Guns N' Roses Sweet Child O' Mine - Classic Rock Song
piano_man_joel = download_songs("https://www.youtube.com/watch?v=QwVjTlTdIDQ", 30) #Bill Joel's Piano Man - A soft rock song
disco_inferno  = download_songs("https://www.youtube.com/watch?v=A_sY2rjxq6M", 0) #The Tramp's Disco Inferno - Classic Disco song
blinding_lights = download_songs("https://www.youtube.com/watch?v=fHI8X4OXluQ", 120) #The Weekend's Blinding Lights - The Current #1 Pop Song
mannish_boy = download_songs("https://www.youtube.com/watch?v=_9IIDE_45Pc", 0) #Muddy Watters Mannish Boy - A classic Blues Song 

#Fancier Examples
arabic_rock = download_songs("https://www.youtube.com/watch?v=xzVZkv7HyDo", 70) #Ramy Essam's Harrara - Arabic rock music
french_dance = download_songs ("https://www.youtube.com/watch?v=SXoQHp2cQQc", 0) #Stromae's Alor En Danse - A french hiphop-dance song
star_wars_cantina = download_songs(" https://www.youtube.com/watch?v=EsvfptdFXf4", 0) #John William's Mad About Me - The cantina song from starwars
so_what_davis = download_songs("https://www.youtube.com/watch?v=ylXk1LBvIqU", 420) #Miles Davis So What - A Jazz Blues song
milestones_davis = download_songs("https://www.youtube.com/watch?v=k94zDsJ-JMU", 120) #Miles Davis Milestones - A Jazz Blues Song


Downloaded: 'Art Blakey & the Jazz Messengers - Moanin'.webm'
Converted into: 'song.mp3'
Successfully Extracted Features

Downloaded: 'Wolfgang Amadeus Mozart - Lacrimosa | Watchmen OST.webm'
Converted into: 'song.mp3'
Successfully Extracted Features

Downloaded: 'Bob Marley & The Wailers - Three Little Birds (Official Video).webm'
Converted into: 'song.mp3'
Successfully Extracted Features

Downloaded: 'Big Iron.webm'
Converted into: 'song.mp3'
Successfully Extracted Features

Downloaded: 'Metallica-Master Of Puppets (Lyrics).webm'
Converted into: 'song.mp3'
Successfully Extracted Features

Downloaded: 'Guns N' Roses - Sweet Child O' Mine (Official Music Video).webm'
Converted into: 'song.mp3'
Successfully Extracted Features

Downloaded: 'Billy Joel - Piano Man (Audio).webm'
Converted into: 'song.mp3'
Successfully Extracted Features

Downloaded: 'The Trammps - Disco Inferno.webm'
Converted into: 'song.mp3'
Successfully Extracted Features

Downloaded: 'The Weeknd - Blinding Lights (Off

In [0]:
supervised_from_youtube( moanin_jazz )
supervised_from_youtube( requiem_d_classic )
supervised_from_youtube( three_little_birds_marley )
supervised_from_youtube( big_iron_country )
supervised_from_youtube( master_of_puppets_metallica )
supervised_from_youtube( sweet_child_roses )
supervised_from_youtube( piano_man_joel )
supervised_from_youtube( disco_inferno )
supervised_from_youtube( blinding_lights )
supervised_from_youtube( mannish_boy )


supervised_from_youtube( arabic_rock )
supervised_from_youtube( french_dance )
supervised_from_youtube( star_wars_cantina )
supervised_from_youtube( milestones_davis )
supervised_from_youtube( so_what_davis )

Supervised Predicted 'Art Blakey & the Jazz Messengers - Moanin'' as Jazz
Guess order is: Jazz, Blues, Disco, Hiphop, Reggae, Rock, Classical, Country, Pop, Metal, 
Supervised Predicted 'Wolfgang Amadeus Mozart - Lacrimosa | Watchmen OST' as Classical
Guess order is: Classical, Jazz, Country, Blues, Hiphop, Rock, Reggae, Pop, Metal, Disco, 
Supervised Predicted 'Bob Marley & The Wailers - Three Little Birds (Official Video)' as Reggae
Guess order is: Reggae, Hiphop, Disco, Jazz, Country, Blues, Pop, Metal, Rock, Classical, 
Supervised Predicted 'Big Iron' as Country
Guess order is: Country, Blues, Reggae, Hiphop, Rock, Jazz, Pop, Disco, Metal, Classical, 
Supervised Predicted 'Metallica-Master Of Puppets (Lyrics)' as Metal
Guess order is: Metal, Hiphop, Rock, Pop, Blues, Disco, Country, Reggae, Jazz, Classical, 
Supervised Predicted 'Guns N' Roses - Sweet Child O' Mine (Official Music Video)' as Rock
Guess order is: Rock, Country, Disco, Metal, Blues, Hiphop, Jazz, Pop, Reggae, Classic

In [0]:
unsupervised_from_youtube( moanin_jazz )
unsupervised_from_youtube( requiem_d_classic )
unsupervised_from_youtube( three_little_birds_marley )
unsupervised_from_youtube( big_iron_country )
unsupervised_from_youtube( master_of_puppets_metallica )
unsupervised_from_youtube( sweet_child_roses )
unsupervised_from_youtube( piano_man_joel )
unsupervised_from_youtube( disco_inferno )
unsupervised_from_youtube( blinding_lights )
unsupervised_from_youtube( mannish_boy )


unsupervised_from_youtube( arabic_rock )
unsupervised_from_youtube( french_dance )
unsupervised_from_youtube( star_wars_cantina )
unsupervised_from_youtube( milestones_davis )
unsupervised_from_youtube( so_what_davis )

Kmeans Group: 9

Kmeans Group: 6

Kmeans Group: 7

Kmeans Group: 6

Kmeans Group: 1

Kmeans Group: 1

Kmeans Group: 9

Kmeans Group: 9

Kmeans Group: 1

Kmeans Group: 7

Kmeans Group: 9

Kmeans Group: 7

Kmeans Group: 6

Kmeans Group: 9

Kmeans Group: 9

